In [6]:
from ptbxlae.dataprocessing.ptbxlDS import PtbxlDM
import pandas as pd
from tqdm.auto import tqdm

In [7]:
# Get latent-space representations
from ptbxlae.modeling.simpleStaticVAE import StaticSimpleVAE
import torch

dm = PtbxlDM(root_folder="../data")
dm.setup(stage="test")
dl = dm.test_dataloader()

m = StaticSimpleVAE.load_from_checkpoint('../cache/savedmodels/last.ckpt').eval()

latent_representations = torch.cat([m.encoder(x.to('cuda')) for x in tqdm(dl)]).cpu().detach().numpy()


/home/isears/VirtualEnvironments/default/lib/python3.11/site-packages/lightning/fabric/utilities/cloud_io.py:56: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


  0%|          | 0/48 [00:00<?, ?it/s]

In [8]:
latent_df = pd.DataFrame(data=latent_representations, columns=[f"latent.{i}" for i in range(0, 40)])
latent_df

,latent.0,latent.1,latent.2,latent.3,latent.4,latent.5,latent.6,latent.7,latent.8,latent.9,...,latent.30,latent.31,latent.32,latent.33,latent.34,latent.35,latent.36,latent.37,latent.38,latent.39
0,0.240925,-0.054612,-0.356227,0.518779,0.011614,-0.385880,0.504324,0.022064,-0.399671,0.455572,...,-0.162054,0.674562,-0.298667,-0.373694,0.580265,-0.149219,-0.425728,0.412733,0.267284,0.019731
1,-0.384866,0.372018,-0.318516,0.273307,0.083985,-0.214989,0.560813,-0.442406,0.319279,0.017509,...,-0.360264,0.513035,-0.253441,0.195727,0.303715,-0.381917,0.459016,-0.255616,0.159404,0.497891
2,0.161677,0.005288,-0.131309,0.042168,0.066826,0.142959,0.196835,-0.291769,0.150100,0.308136,...,-0.058341,0.531657,-0.151961,-0.119552,0.554298,-0.131372,-0.190337,0.423514,0.095751,0.051979
3,-0.095175,-0.592633,0.404260,0.679462,-0.227989,-0.562070,0.424491,0.650812,-0.261457,-0.548412,...,0.609387,0.530620,-0.245669,-0.521058,0.622074,0.562665,-0.264129,-0.525969,0.672711,0.367804
4,0.134826,0.052065,-0.093223,-0.575495,0.109379,0.533943,0.047910,-0.091029,-0.547792,0.031417,...,-0.406537,0.744216,0.114941,0.142103,-0.253589,-0.527717,0.571641,0.281589,0.077397,0.079396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,-0.025572,-0.364828,0.415199,0.501619,-0.308914,-0.098605,0.878385,-0.076170,-0.387869,0.383065,...,-0.156741,-0.407373,0.609749,0.386804,-0.220133,-0.317620,0.797070,0.104670,-0.329617,-0.177884
1501,-0.193013,-0.639917,0.543897,0.508901,-0.066866,-0.560633,-0.174815,0.847065,0.063572,-0.284538,...,0.884667,0.242113,-0.229756,-0.656575,0.361258,0.582131,0.080878,-0.401371,-0.630827,0.817880
1502,0.082895,-0.414079,0.090292,0.532323,-0.603302,0.445887,-0.039943,-0.356129,0.751835,-0.546221,...,-0.006794,0.608117,-0.565292,0.495161,0.128746,-0.417647,0.781852,-0.434869,0.133991,0.669108
1503,-0.320701,0.733520,-0.108950,-0.378671,0.581683,0.052627,-0.411344,0.375157,0.352198,-0.366290,...,0.526301,-0.337563,-0.016175,0.686562,-0.283191,-0.145472,0.796980,-0.257066,-0.330134,0.643052


In [9]:
# Util function to do a quick LR on a specified target based on latent variables
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

def eval_predictive_power_binary_outcome(x, y):
    lr = LogisticRegression()

    ret = {
        "Total usable": len(y),
        "% positive": sum(y) / len(y)
    }

    if sum(y) < 10:  # Won't be able to do CV
        ret['Avg CV score'] = float('nan')
        return ret

    try:
        scores = cross_val_score(lr, x, y, cv=5, scoring='roc_auc')
        ret['Avg CV score'] = sum(scores) / len(scores)
        return ret
    except ValueError:
        ret['Avg CV score'] = float('nan')
        return ret

In [10]:
import ast

dm = PtbxlDM(root_folder="../data")
dm.setup(stage="test")
metadata = dm.test_ds.dataset.metadata.iloc[dm.test_ds.indices].reset_index()

all_scps = pd.read_csv("../data/scp_statements.csv", index_col=0)

def ptb_val_to_pseudobinary_label(scp_code_of_interest: str, this_recording_scp_codes: str):
    if scp_code_of_interest not in this_recording_scp_codes.keys():
        return 0.0
    elif scp_code_of_interest in this_recording_scp_codes.keys() and this_recording_scp_codes[scp_code_of_interest] == 100.0:
        return 1.0
    else:
        return float('nan')

combined_df = pd.concat([metadata, latent_df], axis=1)

results = list()
for scp_code in tqdm(all_scps.index.to_list()):
    combined_df[f'scp.{scp_code}'] = combined_df['scp_codes'].apply(lambda codes: ptb_val_to_pseudobinary_label(scp_code, ast.literal_eval(codes)))
    relevant_df = combined_df[~combined_df[f'scp.{scp_code}'].isna()]

    res = eval_predictive_power_binary_outcome(relevant_df[latent_df.columns], relevant_df[f'scp.{scp_code}'])
    res['Target'] = all_scps.loc[scp_code]['description']
    results.append(res)


results_df = pd.DataFrame.from_records(results)
results_df.nlargest(n=50, columns=['Avg CV score'])


  0%|          | 0/71 [01:20<?, ?it/s]

,Total usable,% positive,Avg CV score,Target
64,1504,0.013963,0.919892,normal functioning artificial pacemaker
15,1505,0.021927,0.867498,complete left bundle branch block
6,1451,0.062715,0.786737,anteroseptal myocardial infarction
8,1502,0.071238,0.779516,left anterior fascicular block
16,1497,0.012692,0.735906,inferolateral myocardial infarction
13,1498,0.014686,0.647704,ischemic in anterolateral leads
4,1333,0.399850,0.631403,normal ECG
9,1491,0.057009,0.628542,non-specific ischemic
1,1496,0.022059,0.614840,non-specific ST changes
24,1505,0.007973,0.573381,left posterior fascicular block
